In [ ]:
import pandas as pd

df = pd.read_csv("data/20241123205432.csv")
df[:3]

In [ ]:
df['factor'] = df['question'].apply(lambda x: 'IB' if x%2 == 0 else 'IH')
df2 = df.drop('base', axis=1).groupby(['feature', 'steerage', 'factor'], as_index=False).mean()
df2 = df2.pivot(index=['feature', 'steerage'],columns='factor', values='mean_score').reset_index()
df2[:3]


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

plt.figure(figsize=(10, 6))

def scatter_plot_feature(feature, group):
    """Group should have IB/IB and be in sorted order"""
    sc = plt.scatter(group['IB'], group['IH'],  marker='o',label=feature)
    color = sc.get_facecolor()
    for i in range(len(group) - 1):
        plt.annotate(
            '',  # No text for the annotation
            xy=(group['IB'].iloc[i + 1], group['IH'].iloc[i + 1]),  # Arrow end
            xytext=(group['IB'].iloc[i], group['IH'].iloc[i]),  # Arrow start
            arrowprops=dict(arrowstyle='->', lw=1.5, color=color),  # Arrow style
        )
    plt.text(group['IB'].iloc[-1] + 0.04, group['IH'].iloc[-1], feature, fontsize=9, ha='left', va='center')

for feature, group in df2.groupby('feature'):
    scatter_plot_feature(feature, group)

# Add labels, legend, and grid
plt.xlim(1, 7)
plt.ylim(1, 7)
plt.xticks(range(1, 8))  # Tick marks from 1 to 7
plt.yticks(range(1, 8))  # Tick marks from 1 to 7
plt.xlabel('IB')
plt.ylabel('IH')
plt.title('Feature steerage effect on Instrumental Harm and Impartial Beneficence')
plt.grid(True)

# Show plot
plt.show()